In [1]:
import pandas as pd


Here we will create a new label into the labels.csv, it will include if the pose is from a men or a women. 

In [3]:
import pandas as pd

# Load labels.csv
df = pd.read_csv("data/labels.csv")

# Function to detect gender based on id
def detect_gender(id_str):
    if id_str.startswith("syn_m"):
        return "male"
    elif id_str.startswith("syn_f"):
        return "female"
    else:
        return "unknown"

# Add new column
df["gender"] = df["id"].apply(detect_gender)

# Save updated CSV
df.to_csv("data/labels_with_gender.csv", index=False)

print(df.head())


                  id  height  chest_circ  waist_circ  hip_circ  thigh_circ  \
0  syn_m000000-0-Pre  177.76       83.58       73.89     96.10       53.34   
1  syn_m000000-0-Pos  177.76       83.58       73.89     96.10       53.34   
2  syn_m000000-1-Pre  177.76       86.36       69.62     96.23       53.46   
3  syn_m000000-1-Pos  177.76       86.36       69.62     96.23       53.46   
4  syn_m000000-2-Pre  177.76       90.95       65.70     96.60       53.72   

   knee_circ  calf_circ  abd_circ  neck_circ  biceps_circ gender  
0      36.45      32.81     74.71      35.53        23.53   male  
1      36.45      32.81     74.71      35.53        23.53   male  
2      36.34      34.71     68.73      35.63        26.03   male  
3      36.34      34.71     68.73      35.63        26.03   male  
4      36.27      36.68     62.95      36.23        28.67   male  
